In [2]:
import sys
sys.path.insert(0, '../../osiris_query/')
from query.results import QueryResult

In [5]:
df_vti = QueryResult.read_pickle('monitor/inschrijfhistorie_2020').frame
df_ooa = QueryResult.read_pickle('monitor/ooa_aanmelding_2020').frame
df_dos = QueryResult.read_pickle('ooa/dos_BA2021_MATCHING_I').frame
df_opl = QueryResult.read_pickle('referentie/ref_OST_OPLEIDING').frame

In [15]:
df_dos = df_dos.loc[df_dos['processtap'].str.contains('O_DATUM')]
df_dos = df_dos.loc[df_dos['gesloten_antwoord_code'] == 'APRIL']
df_dos = df_dos.loc[df_dos['faculteit'] == 'BETA']

In [16]:
cols = ['studentnummer', 'opleiding', 'croho']
df_ooa_c = df_ooa.merge(df_vti[cols], on=['studentnummer', 'opleiding'])

In [17]:
cols = ['ooa_id', 'gesloten_antwoord_code']
df_m = df_dos[cols].merge(df_ooa_c, on=['ooa_id'])

In [18]:
df_m['opleiding'].value_counts()

Series([], Name: opleiding, dtype: int64)

In [19]:
cols = ['studentnummer', 'croho', 'opleiding']
df_inlees = df_m[cols].copy()

In [20]:
df_nast = df_inlees.loc[df_inlees['opleiding'] == 'NAST']['studentnummer']
df_wisk = df_inlees.loc[df_inlees['opleiding'] == 'WISK']['studentnummer']
df_schk = df_inlees.loc[df_inlees['opleiding'] == 'SCHK']['studentnummer']
df_inca = df_inlees.loc[df_inlees['opleiding'] == 'INCA']['studentnummer']
df_inku = df_inlees.loc[df_inlees['opleiding'] == 'INKU']['studentnummer']

In [21]:
stud_nast = df_inlees['studentnummer'].isin(df_nast)
stud_wisk = df_inlees['studentnummer'].isin(df_wisk)
stud_schk = df_inlees['studentnummer'].isin(df_schk)
stud_inca = df_inlees['studentnummer'].isin(df_inca)
stud_inku = df_inlees['studentnummer'].isin(df_inku)

In [22]:
# INKU/INCA
filt = stud_inca & stud_inku
df_inlees.loc[filt & (df_inlees['opleiding'] == 'INKU'), 'VOOR_ACTIVITEIT_BETA_TWIN'] = 'INCA'
df_inlees.loc[filt & (df_inlees['opleiding'] == 'INCA'), 'VOOR_ACTIVITEIT_BETA_TWIN'] = 'INKU'

# WISK/INCA
filt = stud_wisk & stud_inca
df_inlees.loc[filt & (df_inlees['opleiding'] == 'WISK'), 'VOOR_ACTIVITEIT_BETA_TWIN'] = 'INCA'
df_inlees.loc[filt & (df_inlees['opleiding'] == 'INCA'), 'VOOR_ACTIVITEIT_BETA_TWIN'] = 'WISK'

# NAST/SCHK
filt = stud_nast & stud_schk
df_inlees.loc[filt & (df_inlees['opleiding'] == 'SCHK'), 'VOOR_ACTIVITEIT_BETA_TWIN'] = 'NAST'
df_inlees.loc[filt & (df_inlees['opleiding'] == 'NAST'), 'VOOR_ACTIVITEIT_BETA_TWIN'] = 'SCHK'

# NAST/WISK
filt = stud_nast & stud_wisk
df_inlees.loc[filt & (df_inlees['opleiding'] == 'WISK'), 'VOOR_ACTIVITEIT_BETA_TWIN'] = 'NAST'
df_inlees.loc[filt & (df_inlees['opleiding'] == 'NAST'), 'VOOR_ACTIVITEIT_BETA_TWIN'] = 'WISK'

ValueError: cannot set a frame with no defined index and a scalar

In [14]:
df_inlees.query("VOOR_ACTIVITEIT_BETA_TWIN == VOOR_ACTIVITEIT_BETA_TWIN").to_excel('output/inlees_bestand_beta_2020.xlsx', index=False)